In [1]:
import h5py
import torch
from selected_system import mols, mol_file
from ani_benchmark import ANIBenchmark
import pandas
import os
import tqdm
from IPython.display import display
import itertools
tqdm.monitor_interval = 0

/home/gaoxiang/pytorch-dev/pytorchdev/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
torch.set_num_threads(1)
fm = h5py.File(os.path.join('../',mol_file), "r")

benchmarks = {
    'C': ANIBenchmark(device=torch.device("cpu")),
}

if torch.cuda.is_available():
    benchmarks.update({
        'G': ANIBenchmark(device=torch.device("cuda")),
    })

In [3]:
for i in mols:
    print('number of atoms:', i)
    smiles = mols[i]
    for s in smiles:
        print('Running benchmark on molecule', s)
        key = s.replace('/', '_')
        coordinates = torch.from_numpy(fm[key][()])
        coordinates = coordinates[:200]
        species = fm[key].attrs['species'].split()
        results = {}
        for b,m in tqdm.tqdm_notebook(list(itertools.product(benchmarks, ['1','B']))):
            bench = benchmarks[b]
            coordinates = coordinates.type(bench.aev_computer.dtype)
            try:
                if m == '1':
                    result = bench.oneByOne(coordinates, species)
                elif m == 'B':
                    result = bench.inBatch(coordinates, species)
                else:
                    raise ValueError('BUG here')
                result['forward'] = result['aev'] + result['energy']
                result['total'] = result['forward'] + result['force']
            except RuntimeError as e:
                print(e)
                result = {'aev': None, 'energy': None, 'force': None, 'total': None }
            results[b + ',' + m] = result
        df = pandas.DataFrame(results)
        display(df)
        break

number of atoms: 20
Running benchmark on molecule COC(=O)c1ccc([N+](=O)[O-])cc1


,"C,1","C,B","G,1","G,B"
aev,0.700781,0.537888,1.095591,0.022238
energy,0.422604,0.183495,0.564654,0.002251
force,1.253797,0.561175,1.283622,0.010574
forward,1.123385,0.721383,1.660245,0.024489
total,2.377182,1.282558,2.943866,0.035064


number of atoms: 50
Running benchmark on molecule O=[N+]([O-])c1ccc(NN=Cc2ccc(C=NNc3ccc([N+](=O)[O-])cc3[N+](=O)[O-])cc2)c([N+](=O)[O-])c1


,"C,1","C,B","G,1","G,B"
aev,1.680758,2.910254,1.120638,0.033106
energy,0.546913,0.406957,0.474391,0.002537
force,3.637797,2.189924,1.495740,0.036373
forward,2.227671,3.317210,1.595029,0.035643
total,5.865469,5.507135,3.090769,0.072016


number of atoms: 10
Running benchmark on molecule N#CCC(=O)N


...

number of atoms: 100
Running benchmark on molecule CC(C)C[C@@H](C(=O)O)NC(=O)C[C@@H]([C@H](CC1CCCCC1)NC(=O)CC[C@@H]([C@H](Cc2ccccc2)NC(=O)OC(C)(C)C)O)O


,"C,1","C,B","G,1","G,B"
aev,6.933071,11.386394,1.226802,0.118203
energy,1.004400,0.848959,0.517654,0.001794
force,8.206886,6.606211,1.827498,0.108411
forward,7.937472,12.235354,1.744456,0.119997
total,16.144357,18.841565,3.571954,0.228409


number of atoms: 305
Running benchmark on molecule [H]/N=C(/N)\NCCC[C@H](C(=O)N[C@H]([C@@H](C)O)C(=O)N[C@H](Cc1ccc(cc1)O)C(=O)NCCCC[C@@H](C(=O)NCCCC[C@@H](C(=O)NCC(=O)O)NC(=O)[C@H](CCCCNC(=O)[C@@H](Cc2ccc(cc2)O)NC(=O)[C@@H]([C@@H](C)O)NC(=O)[C@@H](CCCN/C(=N\[H])/N)N)NC(=O)[C@@H](Cc3ccc(cc3)O)NC(=O)[C@@H]([C@@H](C)O)NC(=O)[C@@H](CCCN/C(=N\[H])/N)N)NC(=O)[C@@H](Cc4ccc(cc4)O)NC(=O)[C@@H]([C@@H](C)O)NC(=O)[C@@H](CCCN/C(=N\[H])/N)N)N


cuda runtime error (2) : out of memory at /home/gaoxiang/pytorch/aten/src/THC/generic/THCStorage.cu:58



,"C,1","C,B","G,1","G,B"
aev,22.462832,38.629616,1.459434,NaN
energy,2.698251,2.419860,0.525845,NaN
force,21.112948,20.788418,3.065523,NaN
forward,25.161083,41.049476,1.985280,NaN
total,46.274031,61.837895,5.050802,NaN
